# 모두의 말뭉치 - 비출판물 말뭉치
> https://kli.korean.go.kr/corpus/request/corpusList.do

In [34]:
# Hugging Face Transfomers 환경 세팅
!pip install transformers torch
!pip install kobert-transformers

   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ------ --------------------------------- 1.8/10.8 MB 10.1 MB/s eta 0:00:01
   -------------- ------------------------- 3.9/10.8 MB 9.4 MB/s eta 0:00:01
   --------------------- ------------------ 5.8/10.8 MB 9.5 MB/s eta 0:00:01
   ---------------------------- ----------- 7.6/10.8 MB 8.9 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/10.8 MB 8.8 MB/s eta 0:00:01
   -------------------------------------- - 10.5/10.8 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 10.8/10.8 MB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ----------------------------- ---------- 1.8/2.5 MB 9.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 7.6 MB/s eta 0:00:00


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 991.5/991.5 kB 9.2 MB/s eta 0:00:00


In [70]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from collections import Counter
from itertools import chain

In [46]:
# 데이터 로드 
df = pd.read_csv(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\NIKL_NP.csv')
df.head()

,Unnamed: 0,file_id,anno_level,category,title,author_id,author_age,author_occupation,author_sex,author_submission,author_handwriting,text_date,text_subclass,sentence
0,0,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어제는 아내의 생일이었다. 생일을 맞이하여 아침을 준비하겠다고 오전 8시 30분부터...
1,1,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,"주된 메뉴는 스테이크와 낙지볶음, 미역국, 잡채, 소야 등이었다. 스테이크는 자주하..."
2,2,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,그런데 상상도 못한 일이 벌이지고 말았다. 보통 시즈닝이 되지 않은 원육을 사서 스...
3,3,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,앞면을 센불에 1분을 굽고 뒤집는 순간 방부제가 함께 구어진 것을 알았다. 아내의 ...
4,4,WDRW1900100013,원시,비출판물 > 수필,아내의 생일상,P00013,45,직장인/전문직,M,온라인,No,20200000,null_생일상,어처구니 없는 상황이 발생한 것이다. 방부제가 센불에 녹아서 그런지 물처럼 흘러내렸...


In [47]:
# 결측값 확인
df.isnull().sum()

Unnamed: 0             0
file_id                0
anno_level             0
category               0
title                  0
author_id              0
author_age             0
author_occupation     27
author_sex             0
author_submission      0
author_handwriting     0
text_date              0
text_subclass          0
sentence               0
dtype: int64

In [48]:
# 개요 확인
print(f"총 문장 수: {len(df)}")
print('------------------------------------------')
print("title별 분포:")
print(df['title'].value_counts(), '\n')
print('------------------------------------------')
print("문장 길이(문자 수) 통계:")
print(df['sentence'].str.len().describe(), '\n')

총 문장 수: 76805
------------------------------------------
title별 분포:
title
아르반 일지                322
짧은 사랑의 단상             320
Rumex                 295
무제                    287
제목없음                  263
                     ... 
과학과 인문학의 간극에 대한 고찰      1
요즘 나의 생각들               1
출근하기 싫다                 1
독립적인                    1
땅속나라 도둑괴물               1
Name: count, Length: 9448, dtype: int64 

------------------------------------------
문장 길이(문자 수) 통계:
count    76805.000000
mean       114.733312
std        198.525188
min          1.000000
25%         19.000000
50%         44.000000
75%        126.000000
max       5630.000000
Name: sentence, dtype: float64 



In [49]:
# 중복되는 행 있는 지 확인
df.duplicated().sum()

0

## 간단한 TF-IDF + LogisticRegression 분류기 학습/평가 코드 

In [50]:
# 한국어 불용어 리스트 로드
stopwords_kr = set()
with open(r'C:\Users\wooll\OneDrive\문서\GitHub\-\dataset\stopwords-ko.txt') as f:
  for line in f:
    w = line.strip()
    if w:
      stopwords_kr.add(w)    

In [51]:
# 정졔 + 토큰화 함수 (한글 2글자 이상 단어만 추출)
def tokenize_kr(text):
  # 한글/공백 제외 모두 스페이스로
  text = re.sub(r'[^가-힣\s]', ' ', text)
  # 다중 공백 → 단일, 양끝 공백 제거
  text = re.sub(r'\s+', ' ', text).strip()
  # 2글자 이상 한글 단어 추출
  tokens = re.findall(r'[가-힣]{2,}', text)
  # 불용어 제거
  return [t for t in tokens if t not in stopwords_kr]

In [ ]:
# 간단한 감정 사전 정의 (어근 기반)
# 감정 Label: 행복, 슬픔, 분노, 공포, 혐오, 놀람 (처음에 일단 이렇게 정의)
lexicon = {
    '행복': ['행복', '기쁘', '즐거', '환희'],
    '슬픔': ['슬픔', '슬프', '우울', '비통', '눈물'],
    '분노': ['분노', '화나', '열받', '격분'],
    '공포': ['공포', '무섭', '두렵'],
    '혐오': ['혐오', '싫'],
    '놀람': ['놀람', '놀라', '충격']
}

In [ ]:

# 룰 기반 멀티라벨 추출 함수
def extract_emotions_kr(tokens):
    found = set()
    # lexicon: 레이블(문자열) 해당 감정 어근 리스트
    for label, keywords in lexicon.items():
        # 토큰(token) 하나하나에 대해 키워드가 포함되어 있는지 확인
        for tok in tokens:
            for kw in keywords:
                if kw in tok:
                    found.add(label)
                    # 이미 label을 찾았으므로 다음 레이블로 넘어감
                    break
    return list(found)

In [59]:
# 전처리 + 멀티라벨 컬럼 생성
df['tokens']    = df['sentence'].apply(tokenize_kr)
df['emotions']  = df['tokens'].apply(extract_emotions_kr)

In [60]:
# 룰 기반 커버리지 확인
df['has_emotion'] = df['emotions'].apply(lambda lst: bool(lst))
print("라벨링된 문장 비율:", df['has_emotion'].mean())

라벨링된 문장 비율: 0.09244189831391185


In [72]:
# 아직 라벨링 안 된 문장들의 토큰 모으기
unlab_tokens = df.loc[~df['has_emotion'], 'tokens'].tolist()
all_tokens   = list(chain.from_iterable(unlab_tokens))

# 빈도 계산
freq = Counter(all_tokens)

# 상위 500개 혹은 전체 후보를 DataFrame으로
cand_df = pd.DataFrame(freq.most_common(), columns=['token','count'])

# 엑셀로 저장 → 사람이 옆에 emotion 컬럼 달아서 검수
cand_df.to_excel('emotion_token_candidates.xlsx', index=False)

In [ ]:
# 감정사전 재정의 

In [29]:
# 라벨이 있는 문장만 추출
df_lbl = df.dropna(subset=['emotion'])
X, y = df_lbl['sentence'], df_lbl['emotion']

In [30]:
# 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [31]:
# 파이프라인 정의 (TF-IDF + 로지스틱 회귀)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2), min_df=5)),
    ('clf', LogisticRegression(max_iter=1000))
])

In [32]:
# 학습
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(min_df=5, ngram_range=(1, 2))),
                ('clf', LogisticRegression(max_iter=1000))])

In [33]:
# 평가 
y_pred = pipeline.predict(X_test)
print("\n=== 분류기 성능 ===")
print(classification_report(y_test, y_pred, digits=4))


=== 분류기 성능 ===
              precision    recall  f1-score   support

          공포     1.0000    0.1772    0.3011        79
          놀람     0.8143    0.4318    0.5644       132
          분노     0.0000    0.0000    0.0000        27
          슬픔     0.7869    0.3265    0.4615       147
          행복     0.8783    0.5941    0.7088       170
          혐오     0.5237    0.9822    0.6832       337

    accuracy                         0.6177       892
   macro avg     0.6672    0.4186    0.4532       892
weighted avg     0.7040    0.6177    0.5794       892



C:\Users\wooll\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\wooll\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\wooll\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


- 혐오 클래스가 가장 많고, 분노(27), 공포(79) 등은 상대적으로 적음 -> 이로 인해 모델이 혐오를 자주 예측하고 소수 클래스는 예측 못함
- 공포는 예측한건 거의 다 맞았지만 예측을 거의 안했기 때문에 recall = 0.1772로 recall이 매우 낮게 나옴
- 분노(27)은 recall=0.0 으로 한 건도 맞추지 못함

## 

## 간단한 TF-IDF + LogisticRegression 분류기 학습/평가 코드 개선
**개선 방향**
1. 클래스 불균형 대응
  - class_wegiht
  - 오버샘플링/ 언더 샘플링
2. 감정 사전 확장
  - 분노나 공포에 해당하는 어근/키워드 풍부하게 추가
  - 동의어/유의어 사전 (WordNet 한국어 확장) 적용
  - 형태소 분석기로 어간 처리 후 매칭
3. 전처리 고도화 
  - KoNLPy(Mecab 등) 형태소 분석으로 불용어 제거, 어근 일관성 확보
  - n-gram 범위 조절 (ngram_range=(1,3))
  - minimum document frequency (min_df) 하향 조정
4. 모델 변경
  - 위의 전통 ML 한계를 넘어, KoBERT 같은 Transformer 기반 모델을 쓰면
  - 문맥을 반영한 임베딩 → 소수 클래스도 더 잘 구분
  - 적은 데이터에서도 전반적 성능 상승을 기대

In [73]:
# 원본 텍스트 전처리 컬럼 만들기 
